# Churn Prediction for a Telecommunications Company

Customer churn refers to a percentage of customers that stopped purchasing the goods or service of a company during a period of time [Learn More about Customer Churn](https://mailchimp.com/resources/customer-churn/#:~:text=Customer%20churn%20is%20the%20percentage%20of%20customers%20who,is%20something%20that%20all%20businesses%20want%20to%20avoid.). In our case, we are dealing with a telecommunications company. The goal is to predict customers that will stop using the company's service.

## Importing the Necessary Libraries

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Exploratory Data Analysis

In [13]:
df = pd.read_csv("/workspaces/FLiT-Apprenticeship-Data-Science-Projects/Project_5 Churn-Prediction-Model-with-Deployment/Telecoms Company data.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [14]:
#getting the shape of our dataset, i.e, the number of rows and columns
df.shape

(7043, 21)

In [15]:
#getting a summary of our dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Upon further investigation, we can see that some of our columns have the wrong datatype so we will take care of it as we move further. This should be done to ensure easy data visualization and model building.

Only one column has the wrong data type and that is our `TotalCharges` column. We will take care of it in the following steps:
* We investigate why pandas recognized it as an `object` datatype. This could be because a value was wrongly inputted. Once we figure that out and take care of it;
* We will then convert to the appropriate data type using a function in Pandas called the `astype()` function  

In [16]:
#investigation
df["TotalCharges"].value_counts()

TotalCharges
          11
20.2      11
19.75      9
20.05      8
19.9       8
          ..
6849.4     1
692.35     1
130.15     1
3211.9     1
6844.5     1
Name: count, Length: 6531, dtype: int64

Using the `value_counts()` function, we can see that some of the values in that column are empty spaces. This is why pandas recognized the entire column as object. 

In [17]:
#filling the empty spaces with zeros
df["TotalCharges"]= df["TotalCharges"].replace(" ", 0)

In [21]:
#following the next step, we can now change the datatype to the appropriate one
df["TotalCharges"] = df["TotalCharges"].astype("float64")

In [22]:
#lets recheck our dataframe to confirm the conversion
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


Now that is done, we will move forward to perform further data cleaning and analysis

In [26]:
#checking the unqiue values we have in each column in our data frame
for column in df.columns:
    print(column, ":", df[column].nunique())

customerID : 7043
gender : 2
SeniorCitizen : 2
Partner : 2
Dependents : 2
tenure : 73
PhoneService : 2
MultipleLines : 3
InternetService : 3
OnlineSecurity : 3
OnlineBackup : 3
DeviceProtection : 3
TechSupport : 3
StreamingTV : 3
StreamingMovies : 3
Contract : 3
PaperlessBilling : 2
PaymentMethod : 4
MonthlyCharges : 1585
TotalCharges : 6531
Churn : 2


Using a `for loop`, we were able to get the number of unique elements in each of our columns. This will help to perform our
`univariate and bivariate analysis` smoothly.

In [27]:
df.describe() #getting a statistics summary of our dataset

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2279.734304
std,0.368612,24.559481,30.090047,2266.794470
min,0.000000,0.000000,18.250000,0.000000
25%,0.000000,9.000000,35.500000,398.550000
50%,0.000000,29.000000,70.350000,1394.550000
75%,0.000000,55.000000,89.850000,3786.600000
max,1.000000,72.000000,118.750000,8684.800000


## Data Visualizations

### Univariate Analysis

In [28]:
#gender distribution
gender = dict(df.gender.value_counts())
palette = sns.color_palette("mako")
plt.pie(gender.values(), labels = gender.keys(), autopct = "%.0f%%", colrs = palette)
plt.title("Distribution of Gender")
plt.show();

TypeError: pie() got an unexpected keyword argument 'colrs'